## Current Data (Coordiantes)

### L3
- Dock Position: ```(25500, 25500)```
- Transition Position: ```(23000, 26000)```
- Waiting Position: ```(23000, 22000)```
- Elevator Position: ```(23000, 20500)```

### L2
- Dock Position: ```(25500, 25500)```
- Exit Transition Position: ```(23000, 25500)```
- Waiting Position: ```(25500, 25500)```
- Elevator Position: ```(25500, 24000)```

### L1

### B1

### B2

## Process

1. Vacuum moves to the waiting position in L3 via Transition Point
2. Elevator moves to L3 (Thinking on canceling the automation on Mijia, using program to cancel instead)
3. Vacuum enters the elevator using the "Elevator Position" (Until the state of the vacuum is idle, data from Home Assistant API)
4. Elevator moves to the destination level (currently L2)
5. Vacuum exits the elevator using the waiting position of L3
6. Standby for 5-10s and wait the elevator doors to close
7. Activate the remote control mode and stop it after 5 seconds (Or change the map manually through the program)
8. Activate the docking mode then start the cleaning process

In [1]:
import json
import os
import sys

In [ ]:
# positions = json.loads(open('jsons/positions.json', 'r', encoding='utf-8').read())
# type(positions['3']['Dock Position']['x'])

int

In [ ]:
sys.path.extend(['.', '..'])
from mijiaAPI import mijiaLogin

if not os.path.exists('jsons'):
    os.mkdir('jsons')

pass_file = json.load(open('jsons/credentials.json', 'r'))

username = pass_file['username']
password = pass_file['password']

In [ ]:
print(username, password)

In [ ]:
api = mijiaLogin()
auth = api.QRlogin()
with open('jsons/auth_mijia.json', 'w') as f:
    json.dump(auth, f, indent=2)

Scan the QR code below with Mi Home app
Failed to print QR code to terminal, please use the qr.png file in the current directory.


In [ ]:
from mijiaAPI import mijiaDevices, mijiaAPI
import time

api = mijiaAPI(auth)
devices = api.get_devices_list()['list']
with open('jsons/devices.json', 'w') as f:
    json.dump(devices, f, indent=2, ensure_ascii=False)
time.sleep(2)

In [26]:
from mijiaAPI import get_device_info

model = 'giot.switch.v51ksm'
info = get_device_info(model)

if not os.path.exists('dev_info'):
    os.mkdir('dev_info')

with open(f'dev_info/{model}.json', 'w', encoding='utf-8') as f:
    json.dump(info, f, ensure_ascii=False, indent=2)

In [27]:
elevator_name_list = ['负二楼电梯','负一楼电梯','一楼电梯','二楼电梯','三楼电梯']
elevator_did_list = json.loads('{}')

for device in devices:
    if device['name'] in elevator_name_list:
        elevator_did_list[device['name']] = device['did']

with open('jsons/elevator_did.json', 'w') as f:
    json.dump(elevator_did_list, f, indent=2, ensure_ascii=False)

In [10]:
devices = json.loads(open('jsons/devices.json', 'r').read())
model = 'giot.switch.v51ksm'

for device in devices:
    if device['model'] == model:
        print(device['name'], device['did'])

3楼电梯 1138585740
-2楼电梯 1138585779
2楼电梯 1139221639
1楼电梯 1139222448


In [ ]:
did = elevator_did_list['三楼电梯']
dev_info = json.load(open(f'dev_info/{model}.json', 'r'))
device = mijiaDevices(api, dev_info, did=did, sleep_time=2)

print(device.run_action('toggle')) # 点按
# print(device.on)

False


In [12]:
from elevator import Elevator

model = 'giot.switch.v51ksm'
credentials = 'jsons/credentials.json'
auth = 'jsons/auth.json'
devices = 'jsons/devices.json'
elevator_did = 'jsons/elevator_did.json'
dev_info = f'dev_info/{model}.json'


elevator = Elevator(credentials=credentials, auth=auth, devices=devices, elevator_did=elevator_did, dev_info=dev_info)
print(elevator.model)

giot.switch.v51ksm


In [ ]:
print(elevator.model)
elevator.refresh_device_list()
elevator.refresh_floor_list()

'''
 3 0
 2 1
 1 2
-1 3 

'''

giot.switch.v51ksm
